# Clean up

Lo que se pretende es aplicar los conceptos vistos de "Análisis Exploratorio" para limpiar los distintos set de datos proporcionados.

Archivos en: https://www.kaggle.com/benhamner/sf-bay-area-bike-share

Los archivos disponibles son 4:
    * station.csv (8 KB)
    * weather.csv (428 KB)
    * trip.csv    (77 M)
    * status.csv  (1,9 GB)

In [10]:
import pandas as pd
import dateutil

## Station.csv

In [46]:
station = pd.read_csv('station.csv', sep=',', parse_dates=['installation_date'],
                      infer_datetime_format=True,low_memory=False)

### Types

In [47]:
station.dtypes

id                            int64
name                         object
lat                         float64
long                        float64
dock_count                    int64
city                         object
installation_date    datetime64[ns]
dtype: object

### Null values

In [34]:
station.isnull().any()

id                   False
name                 False
lat                  False
long                 False
dock_count           False
city                 False
installation_date    False
dtype: bool

### Size

In [35]:
station.shape

(70, 7)

### Sample

In [38]:
station.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,2013-08-06
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,2013-08-05
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,2013-08-06
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,2013-08-05
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,2013-08-07


## Weather.csv

In [94]:
weather = pd.read_csv('weather.csv', sep=',', parse_dates=['date'],
                      infer_datetime_format=True,low_memory=False)

### Types

In [95]:
weather.dtypes

date                              datetime64[ns]
max_temperature_f                        float64
mean_temperature_f                       float64
min_temperature_f                        float64
max_dew_point_f                          float64
mean_dew_point_f                         float64
min_dew_point_f                          float64
max_humidity                             float64
mean_humidity                            float64
min_humidity                             float64
max_sea_level_pressure_inches            float64
mean_sea_level_pressure_inches           float64
min_sea_level_pressure_inches            float64
max_visibility_miles                     float64
mean_visibility_miles                    float64
min_visibility_miles                     float64
max_wind_Speed_mph                       float64
mean_wind_speed_mph                      float64
max_gust_speed_mph                       float64
precipitation_inches                      object
cloud_cover         

In [84]:
weather['precipitation_inches'].unique()

array(['0', '0.23', 'T', '0.01', '0.28', '0.63', '0.29', '0.06', '0.85',
       '0.09', '0.64', '0.42', '0.35', '0.43', '0.22', '0.74', '0.03',
       '0.12', '0.16', '0.49', '0.17', '0.08', '0.04', '0.53', '0.07',
       '0.02', '0.83', '1.06', '1.71', '0.37', '0.27', '0.45', '0.78',
       '0.88', '0.66', '0.47', '0.1', '0.61', '0.14', '0.05', '0.68',
       '0.97', '0.26', '0.15', '0.87', '0.57', '0.69', '0.32', '0.21',
       '0.24', '0.52', '0.36', '0.33', '0.25', '0.11', '0.2', '1.18',
       '1.43', '3.12', '0.48', '0.19', '1.09', '0.65', '0.13', '0.91',
       '0.99', '0.18', '0.4', '1.07', nan, '0.41', '0.34', '1.25', '1.85',
       '3.36', '0.71', '1.3', '0.72', '0.6', '0.51', '1.2', '1.28', '3.23',
       '0.55', '1.26', '0.39'], dtype=object)

Llamativamente, "T", es un dato válido, por "trace", significa que se detectó lluvia, pero no la suficiente para poder ser medida.

[Fuente 1](http://help.wunderground.com/knowledgebase/articles/656875-what-does-t-stand-for-on-the-rain-precipitation)

Aquí, [Fuente 2](http://www.experts123.com/q/what-does-the-t-mean-in-the-precipitation-column-of-the-data-listing.html) indica que la precipitación debe ser menor a 0,01 pulgadas

En principio, opino que deberíamos mantener ese valor 'T', ya veremos si nos resulta de utilidad

In [80]:
weather[weather['precipitation_inches'] == 'T']['events'].unique()

array(['Fog', 'Rain', nan, 'Fog-Rain', 'Rain-Thunderstorm'], dtype=object)

Efectivamente, los eventos muestran que fueron días de lluvia, o al menos de humedad debido a la presencia de niebla.

In [58]:
weather['min_dew_point_f'] = pd.to_numeric(weather['min_dew_point_f'], errors='coerce')

In [98]:
weather['events'].unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'Rain-Thunderstorm'], dtype=object)

In [97]:
# Uniformizando

weather['events'] = weather['events'].apply(lambda x: 'Rain' if x == 'rain' else x)

### Null values

In [99]:
weather.isnull().any()

date                              False
max_temperature_f                  True
mean_temperature_f                 True
min_temperature_f                  True
max_dew_point_f                    True
mean_dew_point_f                   True
min_dew_point_f                    True
max_humidity                       True
mean_humidity                      True
min_humidity                       True
max_sea_level_pressure_inches      True
mean_sea_level_pressure_inches     True
min_sea_level_pressure_inches      True
max_visibility_miles               True
mean_visibility_miles              True
min_visibility_miles               True
max_wind_Speed_mph                 True
mean_wind_speed_mph                True
max_gust_speed_mph                 True
precipitation_inches               True
cloud_cover                        True
events                             True
wind_dir_degrees                   True
zip_code                          False
dtype: bool

### Size

In [85]:
weather.shape

(3665, 24)